# **Methylation Biomarkers for Predicting Cancer**

## **Deep Learning Approaches for Cancer Classification - Clinical Data + RF Features**

**Author:** Meg Hutch

**Date:** March 9, 2020

**Objective:** Use neural networks to classify cancer type. 

**Note:** In this version, I will only test the ability of methylation levels to classify cancer types. I will not include phenotypic data for now. Additionally, this version has our data split 70% for training and 30% for testing. The 70% training data will undergo leave-one-out-cross-fold validation to tune hyperparameters prior to testing final performance on the 30% test set. 

Note: This is the new version of the script where we normalize gene counts using DEseq2 in the initial pre-processing script in R. This provided more than double the number of Principal Components that make up 90% of the variance (157). Regardless, we will begin running the deep learning classifier on the revised data. 

Update: This script will include the clinical data

Note: This is on the data of the GI cancers. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

In [4]:
# set working directory for git hub
import os
os.chdir('/home/mrh1996/Multi_Cancer_DL/')
#os.chdir('/projects/p31049/Multi_Cancer_DL/')
#os.chdir('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/')
os. getcwd()

'/home/mrh1996/Multi_Cancer_DL'

**Import Training, Testing, and Principal component data**

**Full Dataset**

In [11]:
# Training set
mcTrain_x = pd.read_csv('02_Processed_Data/GI_Datasets/mcTrain_x_gi_Full_70_30.csv')
mcTrain_y = pd.read_csv('02_Processed_Data/GI_Datasets/mcTrain_y_gi_Full_70_30.csv')
# Testing set
mcTest_x = pd.read_csv('02_Processed_Data/GI_Datasets/mcTest_x_gi_Full_70_30.csv')
mcTest_y = pd.read_csv('02_Processed_Data/GI_Datasets/mcTest_y_gi_Full_70_30.csv')

# Random Forest Features
rf_feats = pd.read_csv('02_Processed_Data/GI_Datasets/rf_100feats_FULL_gi_70_30.csv')

In [ ]:
#mcTrain_y.head()
#pca_Train.head()
#rf_feats.head()

# **Pre-Process Data**

In [12]:
# rename the first column name of the rf_feats dataframes
rf_feats.rename(columns={'Unnamed: 0':'Gene'}, inplace=True)

**Remove the Importance Column**

In [13]:
rf_feats = rf_feats.drop(columns=["0"])

**Convert id to index**

In [14]:
mcTrain_x = mcTrain_x.set_index('id')
mcTrain_y = mcTrain_y.set_index('id')

mcTest_x = mcTest_x.set_index('id')
mcTest_y = mcTest_y.set_index('id')

**Create seperate DF with only Clinical Variables**

In [15]:
mcTrain_clinical_x = mcTrain_x[['dilute_library_concentration', 'age', 'gender', 'frag_mean']]
mcTest_clinical_x = mcTest_x[['dilute_library_concentration', 'age', 'gender', 'frag_mean']]

**Keep only the Genes that were in the rf_feats**

In [16]:
# Create a list of the Genes to keep
rf_genes = rf_feats.Gene
# convert to df
rf_genes = pd.DataFrame(rf_genes)
# set Gene as an index
rf_genes = rf_genes.set_index('Gene')
# Create a list of the gene names
rf_genes = list(rf_genes.index)
rf_genes

['SOX9',
 'COA7',
 'GABRB3',
 'ESRRG',
 'ADGRG5',
 'ALG1L',
 'NPNT',
 'APOE',
 'SERINC5',
 'MEGF9',
 'FUT9',
 'CCM2',
 'ARMC6',
 'OR9Q1',
 'SNX10',
 'TUBB1',
 'HORMAD1',
 'RBM23',
 'GPATCH2L',
 'GABRA2',
 'BAG5',
 'ANO2',
 'NOVA1',
 'ADAM18',
 'CARD11',
 'KCNB2',
 'CYB5A',
 'RAF1',
 'PPP2R5C',
 'ITK',
 'PKP4',
 'CD247',
 'NPSR1',
 'USF2',
 'TACR1',
 'PPP1CB',
 'PAFAH1B1',
 'ARHGAP31',
 'SP1',
 'SEMA3E',
 'CDH7',
 'MMADHC',
 'ANKRD13A',
 'CHKA',
 'PCLO',
 'ATP5A1',
 'GULP1',
 'FAT4',
 'SLC7A4',
 'RFC2',
 'SLC25A42',
 'LZTFL1',
 'KPNA6',
 'CASP8',
 'PLA2G5',
 'ARHGEF7',
 'CAPZB',
 'RASSF9',
 'STK32B',
 'G3BP2',
 'GALM',
 'DIRC3',
 'EEF1E1-BLOC1S5',
 'TMEM202',
 'G6PC',
 'FYCO1',
 'SLC10A2',
 'ETHE1',
 'RPRD2',
 'RBM14-RBM4',
 'CDH12',
 'ACAA1',
 'KDR',
 'CFAP69',
 'FAR2',
 'CCDC39',
 'KCNH6',
 'CLIC5',
 'DIO2',
 'CPD',
 'BMP8B',
 'KANSL2',
 'GPX6',
 'CEBPG',
 'AFMID',
 'GATAD2B',
 'ADTRP',
 'CUBN',
 'HCRTR2',
 'SORL1',
 'UBE2D2',
 'CS',
 'TMBIM6',
 'FBXO44',
 'MYCT1',
 'LMO7DN',
 'CTNNA2

In [17]:
mcTrain_x = mcTrain_x[rf_genes]
mcTest_x = mcTest_x[rf_genes]
mcTrain_x

,SOX9,COA7,GABRB3,ESRRG,ADGRG5,ALG1L,NPNT,APOE,SERINC5,MEGF9,...,UBE2D2,CS,TMBIM6,FBXO44,MYCT1,LMO7DN,CTNNA2,CTAGE6,KCNC2,CGN
id,,,,,,,,,,,,,,,,,,,,,
1,7.423863,151.658908,330.892162,900.408480,529.215349,93.328559,230.139741,26.513795,2198.523885,1398.867826,...,1407.352241,151.658908,623.604459,133.629527,170.748840,77.420282,1806.119718,4.242207,189.838772,196.202083
2,6.389534,134.819163,238.968563,679.846394,456.212712,81.786032,202.548221,34.503482,2031.871741,1340.524186,...,1283.657336,163.572065,675.373720,141.208696,181.462759,83.063939,1403.780571,10.223254,134.819163,243.441237
4,9.820715,127.669301,305.845138,934.370927,503.662407,58.924293,291.815545,22.447350,2016.052587,1359.467611,...,1379.109042,136.087057,627.122830,119.251545,256.741561,98.207155,1791.579091,4.208878,152.922569,193.608390
8,19.686016,122.748097,312.660248,867.342688,522.258414,67.164053,288.342229,30.108024,1961.653557,1391.917105,...,1374.547091,138.960110,639.216507,107.694085,267.498212,118.116094,1642.045303,8.106006,167.910133,245.496195
9,13.628550,106.931701,354.342302,771.585605,543.045304,64.997700,300.876452,25.160400,1962.511213,1414.224160,...,1243.343108,115.318501,624.816604,87.013051,294.586352,88.061401,1956.221113,13.628550,199.186501,172.977751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,14.364680,116.876264,294.475949,852.086726,494.275595,74.435162,214.164327,28.729361,1804.073275,1268.662459,...,1354.197602,131.240944,626.169479,114.917444,287.293609,86.841023,1578.808969,12.405860,170.417345,199.146706
205,9.128777,146.669020,304.292572,791.160688,542.857949,74.855973,247.694154,20.083310,1938.952270,1416.786216,...,1368.707990,108.936741,600.064952,125.368540,252.562835,91.896357,1697.952553,3.042926,161.883648,178.315447
225,10.184947,144.156168,279.694304,779.540146,522.566108,76.778828,233.470315,28.204468,1898.317360,1262.933382,...,1550.462259,130.053934,608.746425,113.601328,228.769570,93.231435,1528.525451,8.618032,160.608774,202.915475


In [18]:
# merge PCs with clinical/phenotypic data
mcTrain_x = pd.merge(mcTrain_clinical_x, mcTrain_x, how="left", on="id") 
mcTest_x = pd.merge(mcTest_clinical_x, mcTest_x, how="left", on="id") 

**Drop the library concentraion (should have already controlled for this perhaps with deseq2 normalization?); Just simply keep demographic data for these experiments!**

In [19]:
mcTrain_x = mcTrain_x.drop(columns=["dilute_library_concentration"])
mcTest_x = mcTest_x.drop(columns=["dilute_library_concentration"])

**Normalize Data**

From my reading, it seems that normalization, as opposed to standardization, is the more optimal approach when data is not normally distributed. 

Normalization will rescale our values into range of [0,1]. We need to normalize both the training and test sets

In [20]:
from sklearn.preprocessing import MinMaxScaler

# The normalization function to be performed will convert dataframe into array, for this reason we'll have to convert it back
# Thus, need to store columns and index
# select all columns
cols = list(mcTrain_x.columns.values)
index_train = list(mcTrain_x.index)
index_test = list(mcTest_x.index)

# Normalize data
scaler = MinMaxScaler()
mcTrain_x = scaler.fit_transform(mcTrain_x.astype(np.float))
mcTest_x = scaler.fit_transform(mcTest_x.astype(np.float))

# Convert back to dataframe
mcTrain_x = pd.DataFrame(mcTrain_x, columns = cols, index = index_train)
mcTest_x = pd.DataFrame(mcTest_x, columns = cols, index = index_test)

# Construct & Run Neural Network

In [22]:
# Import PyTorch packages
import torch
from torch import nn
#from torchvision import datasets, transforms
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [23]:
# define list for results
results_ls = []

# Where we will store correct/incorrect classifications
incorrect_ls = []
correct_ls = []

# Leave-one-out-cross-fold validation function - the for loop will iterate through the dataset, removing one sample (patient)
# at a time in order to create k training and test datasets (where k = number of total samples) always with one sample missing
for index in range (0, 205):
    mcTrain_xy_drop = mcTrain_x.drop(mcTrain_x.index[index]) # add 'drop'suffix so we can differentiate the df with index and the array that will be created in next line
    mcTrain_xy = np.array(mcTrain_xy_drop, dtype = "float32")
    
    # y - target/outputs
    mcTrain_yz_drop = mcTrain_y.drop(mcTrain_y.index[index]) 
    mcTrain_yz = np.array(mcTrain_yz_drop, dtype = "float32")
    
    # reformat into tensors
    xb = torch.from_numpy(mcTrain_xy)
    yb = torch.from_numpy(mcTrain_yz)
    
    # squeeze - function is used when we want to remove single-dimensional entries from the shape of an array.
    yb = yb.squeeze(1) 
    
    # subset the equivalent test set
    mcTrain_test_x_drop = mcTrain_x.iloc[[index]] # add 'drop'suffix so we can differentiate the df with index and the array that will be created in next line
    mcTrain_test_x = np.array(mcTrain_test_x_drop, dtype = "float32")
            
    # y - targets/outputs
    mcTrain_test_y_drop = mcTrain_y.iloc[[index]]
    mcTrain_test_y = np.array(mcTrain_test_y_drop, dtype = "float32")
        
    # Convert arrays into tensors
    test_xb = torch.from_numpy(mcTrain_test_x)
    test_yb = torch.from_numpy(mcTrain_test_y)
    
    # Define the batchsize
    batch_size = 32

    # Combine the arrays
    trainloader = TensorDataset(xb, yb)
    
    # Training Loader
    trainloader = DataLoader(trainloader, batch_size, shuffle=True)
    
    ## Build the Model and define hyperparameters
    
    # summarize experiment with changed parameters
    summary = ('Hidden Layers: 50, LR: 0.10, Epochs: 100, wd: 0.01')
    
    # Define the model with hidden layers
    model = nn.Sequential(nn.Linear(103, 50),
                          nn.ReLU(),
                          nn.Linear(50, 7))
                      
    # Set Stoachastic Gradient Descent Optimizer and the learning rate
    #optimizer = optim.SGD(model.parameters(), lr=0.003)

    # Set Adam optimizer: similar to stochastic gradient descent, but uses momentum which can speed up the actual fitting process, and it also adjusts the learning rate for each of the individual parameters in the model
    optimizer = optim.Adam(model.parameters(), lr=0.10,  weight_decay=0.01) # we can also change momentum parameter

    # loss function
    criterion = nn.CrossEntropyLoss() #don't use with softmax or sigmoid- PyTorch manual indicates "This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class."
    
    # Set epochs - number of times the entire dataset will pass through the network
    epochs = 100
    for e in range(epochs):
        # Define running loss as 0
        running_loss = 0
        
        # Run the model for each xb, yb in the trainloader. For the number of epochs specified, the 
        for xb, yb in trainloader:
            # clear gradients - otherwise they are stored
            optimizer.zero_grad()
            # Training pass
            output = model.forward(xb)
            # caluclate loss calculated from the model output compared to the labels
            loss = criterion(output, yb.long()) 
            # backpropagate the loss
            loss.backward()
            # step function to update the weights
            optimizer.step()
        
            running_loss += loss.item() # loss.item() gets the scalar value held in the loss. 
            # += function: Adds the running_loss (0) with loss.item and assigns back to running_loss
        #else:
        #    print("Epoch {}/{}, Training loss: {:.5f}".format(e+1, epochs, running_loss/len(trainloader)))

    # Apply the model to the testing dataset
    # Thus will enable us to see the predictions for each class
    ps = model(test_xb)
    #print('Network Probabilities', ps)
    
    # Obtain the top prediction
    top_p, top_class = ps.topk(1, dim=1)
    #print('top prediction', top_p)
    #print('true vals', test_yb[:10])
        
    # Drop the grad by using detach
    top_p = top_p.detach().numpy()
    top_class = top_class.detach().numpy()

    # convert to integers
    top_class = top_class.astype(np.int)
    test_yb = test_yb.numpy()
    test_yb = test_yb.astype(np.int)
    
    #print('top class', top_class[:10])
    #print('prediction:', top_class)
    #print('true:', test_yb)
                
    # compare top_class to test_yb
    if top_class == test_yb:                
        results = 1 # prediction and true value are equal
    else: 
        results = 0
    
    # Create if-else statements to identify which classes are being classified correctly/incorrectly
    if results == 0:
        incorrect = test_yb
    else: 
        incorrect = np.array([[999]], dtype=int)
        
    if results == 1:
        correct = test_yb
    else: 
        correct = np.array([[999]], dtype=int)
    #print('Results:', results)
    
    results_ls.append(results)
    incorrect_ls.append(incorrect)
    correct_ls.append(correct)
    #print(results_ls) 

KeyboardInterrupt: 

# **Determine LOOCV Mean Error**

In [ ]:
percent_correct = sum(results_ls)
percent_correct = percent_correct/len(mcTrain_y)*100
percent_incorrect = 100 - percent_correct
percent_incorrect = round(percent_incorrect, 1)
#print('Percent Error', round(percent_incorrect, 1))

# **Incorrect Predictions**

In [ ]:
## Remove the correct elements from the ls to faciliate transforming this list into a dataframe
# First, concatenate all incorrect list elements and format into dataframe
incorrect_res = np.concatenate(incorrect_ls)
incorrect_res = pd.DataFrame(incorrect_res)
incorrect_res.columns = ['diagnosis']
incorrect_res = incorrect_res[incorrect_res.diagnosis != 999] # 999 are the results that were correct - we remove these

# Count number of incorrect predictions by diagnosis
incorrect_pred = incorrect_res.groupby(['diagnosis']).size()
incorrect_pred = pd.DataFrame(incorrect_pred)
incorrect_pred.columns = ['Count']

# Convert the index to the first column and change the numebr to categorical variables
incorrect_pred.reset_index(level=0, inplace=True)
incorrect_pred['diagnosis'] = incorrect_pred['diagnosis'].map({0: 'HEA', 1: 'CRC', 2: 'ESCA', 3: 'HCC', 4: 'STAD'})

# Add a column with the number of cases in each class
mcTrain_y['diagnosis'] = mcTrain_y['diagnosis'].map({0: 'HEA', 1: 'CRC', 2: 'ESCA', 3: 'HCC', 4: 'STAD'})
class_size = mcTrain_y.groupby(['diagnosis']).size()
class_size = pd.DataFrame(class_size)
class_size.columns = ['Sample_n']

# bind class_size to the pred df diagnoses
incorrect_pred = pd.merge(incorrect_pred, class_size, how="left", on="diagnosis") 

# Calculate the percent error for each class
incorrect_pred['Count_Perc_Incorrect'] = incorrect_pred['Count']/incorrect_pred['Sample_n']
incorrect_pred['Count_Perc_Incorrect'] = incorrect_pred['Count_Perc_Incorrect'].multiply(100)

# **Correct Predictions**

In [ ]:
## Remove the incorrect elements from the ls to faciliate transforming this list into a dataframe
# First, concatenate all incorrect list elements and format into dataframe
correct_res = np.concatenate(correct_ls)
correct_res = pd.DataFrame(correct_res)
correct_res.columns = ['diagnosis']
correct_res = correct_res[correct_res.diagnosis != 999] # 999 are the results that were incorrect - we remove these

# Count number of correct predictions by diagnosis
correct_pred = correct_res.groupby(['diagnosis']).size()
correct_pred = pd.DataFrame(correct_pred)
correct_pred.columns = ['Count']

# Convert the index to the first column and change the numebr to categorical variables
correct_pred.reset_index(level=0, inplace=True)
correct_pred['diagnosis'] = correct_pred['diagnosis'].map({0: 'HEA', 1: 'CRC', 2: 'ESCA', 3: 'HCC', 4: 'STAD'})

# Add a column with the number of cases in each class
class_size = mcTrain_y.groupby(['diagnosis']).size()
class_size = pd.DataFrame(class_size)
class_size.columns = ['Sample_n']

# bind class_size to the pred df diagnoses
correct_pred = pd.merge(correct_pred, class_size, how="left", on="diagnosis") 

# Calculate the percent correct for each class
correct_pred['Count_Perc_Correct'] = correct_pred['Count']/correct_pred['Sample_n']
correct_pred['Count_Perc_Correct'] = correct_pred['Count_Perc_Correct'].multiply(100)

# **Save Predictions**

In [ ]:
# convert float to string in order to save as a txt file
percent_error = '     Percent Error'
percent_error = str(percent_error)
percent_incorrect = str(percent_incorrect)

# save the pre-specified summary of the experiment which includes parameter spe$
summary = summary + percent_error + percent_incorrect

# Print Results
print(summary)
print(correct_pred)